# Instance Normalization - Text Data

## Giới thiệu

Instance Normalization cho dữ liệu text với shape (batch, sequence_length, features).

Instance Normalization sẽ normalize theo:
- Mean và variance được tính trên sequence dimension
- Normalize cho từng sample và từng feature độc lập

## Công thức

```
mean = mean(x, dim=1)  # Tính mean theo sequence dim
var = var(x, dim=1)    # Tính variance theo sequence dim
x_norm = (x - mean) / sqrt(var + eps)
output = gamma * x_norm + beta
```

Trong đó:
- gamma: learnable scale parameter (một cho mỗi feature)
- beta: learnable shift parameter (một cho mỗi feature)
- eps: small constant để tránh chia cho 0

Khác với BatchNorm và LayerNorm: InstanceNorm normalize từng feature riêng biệt cho từng sample.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class InstanceNorm1D:
    """
    Instance Normalization cho 1D text/sequence data (3D tensor: batch, sequence_length, features)
    """
    def __init__(self, num_features, eps=1e-5):
        self.num_features = num_features
        self.eps = eps
        
        # Learnable parameters
        self.gamma = np.ones(num_features)  # Scale
        self.beta = np.zeros(num_features)  # Shift
        
    def forward(self, x):
        """
        x shape: (batch, sequence_length, features)
        """
        # Tính mean và var theo sequence dimension
        # Normalize cho từng sample và từng feature độc lập
        mean = np.mean(x, axis=1, keepdims=True)
        var = np.var(x, axis=1, keepdims=True)
        
        # Normalize
        x_norm = (x - mean) / np.sqrt(var + self.eps)
        
        # Scale and shift
        gamma = self.gamma.reshape(1, 1, -1)
        beta = self.beta.reshape(1, 1, -1)
        output = gamma * x_norm + beta
        
        return output

print("InstanceNorm1D class đã được định nghĩa!")


## Ví dụ 1: 1 Sample, 1 Feature

Shape: (1, 5, 1) - 1 batch, sequence length 5, 1 feature


In [ ]:
# Ví dụ 1: 1 Sample, 1 Feature
np.random.seed(42)
x1 = np.random.randn(1, 5, 1) * 10 + 5
print(f"Input shape: {x1.shape}")
print(f"Input mean: {np.mean(x1[0, :, 0]):.4f}, std: {np.std(x1[0, :, 0]):.4f}")

in1 = InstanceNorm1D(num_features=1)
output1 = in1.forward(x1)
print(f"\nOutput mean: {np.mean(output1[0, :, 0]):.4f}, std: {np.std(output1[0, :, 0]):.4f}")

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(x1[0, :, 0], marker='o')
axes[0].set_title('Input (Before IN)')
axes[0].grid(True)
axes[1].plot(output1[0, :, 0], marker='o', color='orange')
axes[1].set_title('Output (After IN)')
axes[1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 2: 1 Sample, 2 Features

Shape: (1, 5, 2) - 1 batch, sequence length 5, 2 features


In [ ]:
# Ví dụ 2: 1 Sample, 2 Features
np.random.seed(42)
x2 = np.random.randn(1, 5, 2) * 10 + 5
print(f"Input shape: {x2.shape}")
print(f"Feature 0 mean: {np.mean(x2[0, :, 0]):.4f}, std: {np.std(x2[0, :, 0]):.4f}")
print(f"Feature 1 mean: {np.mean(x2[0, :, 1]):.4f}, std: {np.std(x2[0, :, 1]):.4f}")

in2 = InstanceNorm1D(num_features=2)
output2 = in2.forward(x2)
print(f"\nFeature 0 - Output mean: {np.mean(output2[0, :, 0]):.4f}, std: {np.std(output2[0, :, 0]):.4f}")
print(f"Feature 1 - Output mean: {np.mean(output2[0, :, 1]):.4f}, std: {np.std(output2[0, :, 1]):.4f}")

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x2[0, :, 0], marker='o')
axes[0, 0].set_title('Feature 0 Input')
axes[0, 0].grid(True)
axes[0, 1].plot(x2[0, :, 1], marker='o', color='green')
axes[0, 1].set_title('Feature 1 Input')
axes[0, 1].grid(True)
axes[1, 0].plot(output2[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Feature 0 Output')
axes[1, 0].grid(True)
axes[1, 1].plot(output2[0, :, 1], marker='o', color='red')
axes[1, 1].set_title('Feature 1 Output')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 3: 2 Samples, 1 Feature

Shape: (2, 5, 1) - 2 batches, sequence length 5, 1 feature


In [ ]:
# Ví dụ 3: 2 Samples, 1 Feature
np.random.seed(42)
x3 = np.random.randn(2, 5, 1) * 10 + 5
print(f"Input shape: {x3.shape}")
print(f"Sample 0 mean: {np.mean(x3[0, :, 0]):.4f}, std: {np.std(x3[0, :, 0]):.4f}")
print(f"Sample 1 mean: {np.mean(x3[1, :, 0]):.4f}, std: {np.std(x3[1, :, 0]):.4f}")

in3 = InstanceNorm1D(num_features=1)
output3 = in3.forward(x3)
print(f"\nSample 0 - Output mean: {np.mean(output3[0, :, 0]):.4f}, std: {np.std(output3[0, :, 0]):.4f}")
print(f"Sample 1 - Output mean: {np.mean(output3[1, :, 0]):.4f}, std: {np.std(output3[1, :, 0]):.4f}")

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x3[0, :, 0], marker='o')
axes[0, 0].set_title('Sample 0 Input')
axes[0, 0].grid(True)
axes[0, 1].plot(x3[1, :, 0], marker='o', color='green')
axes[0, 1].set_title('Sample 1 Input')
axes[0, 1].grid(True)
axes[1, 0].plot(output3[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Sample 0 Output')
axes[1, 0].grid(True)
axes[1, 1].plot(output3[1, :, 0], marker='o', color='red')
axes[1, 1].set_title('Sample 1 Output')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 4: 2 Samples, 2 Features

Shape: (2, 5, 2) - 2 batches, sequence length 5, 2 features


In [ ]:
# Ví dụ 4: 2 Samples, 2 Features
np.random.seed(42)
x4 = np.random.randn(2, 5, 2) * 10 + 5
print(f"Input shape: {x4.shape}")

in4 = InstanceNorm1D(num_features=2)
output4 = in4.forward(x4)

for b in range(2):
    for f in range(2):
        print(f"Sample {b}, Feature {f} - Input mean: {np.mean(x4[b, :, f]):.4f}, std: {np.std(x4[b, :, f]):.4f}")
        print(f"Sample {b}, Feature {f} - Output mean: {np.mean(output4[b, :, f]):.4f}, std: {np.std(output4[b, :, f]):.4f}")

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for b in range(2):
    for f in range(2):
        axes[b, f*2].plot(x4[b, :, f], marker='o')
        axes[b, f*2].set_title(f'Sample {b}, Feat {f} Input')
        axes[b, f*2].grid(True)
        axes[b, f*2+1].plot(output4[b, :, f], marker='o', color='orange')
        axes[b, f*2+1].set_title(f'Sample {b}, Feat {f} Output')
        axes[b, f*2+1].grid(True)
plt.tight_layout()
plt.show()
